In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor

In [2]:
df_itm_cat = pd.read_csv('00-Input/item_categories.csv')
df_itm = pd.read_csv('00-Input/items.csv')
df_sh = pd.read_csv('00-Input/shops.csv')
df_tr = pd.read_csv('00-Input/sales_train.csv')
df_te = pd.read_csv('00-Input/test.csv')

In [3]:
df_tr

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [4]:
#df_tr_sum = df_tr.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day':['sum']})
#df_tr_sum.reset_index()

,date_block_num,shop_id,item_id,item_cnt_day
,,,,sum
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1609119,33,59,22087,6.0
1609120,33,59,22088,2.0
1609121,33,59,22091,1.0


In [6]:
train_set = df_tr.drop(columns=['date','item_price'])
train_set

,date_block_num,shop_id,item_id,item_cnt_day
0,0,59,22154,1.0
1,0,25,2552,1.0
2,0,25,2552,-1.0
3,0,25,2554,1.0
4,0,25,2555,1.0
...,...,...,...,...
2935844,33,25,7409,1.0
2935845,33,25,7460,1.0
2935846,33,25,7459,1.0
2935847,33,25,7440,1.0


In [31]:
train_set = df_tr.drop(columns=['date', 'date_block_num', 'item_price'])

In [32]:
y= train_set['item_cnt_day']
x= train_set.drop(columns=['item_cnt_day'])
print(len(x.columns))

2


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

2055094
880755


In [34]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm

linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

#make predictions using linear regression
price_predict =(linear_model.predict(X_test))

#Measure Performance 
linear_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared error: ",linear_mse)
linear_SqMse = np.sqrt(linear_mse)
print("Root Mean Squared error: ", linear_SqMse)

print("Explain variance score for Linear Regression =", round(sm.explained_variance_score(Y_test,price_predict),2)) 
print("R2 score for Linear Regression =", round(sm.r2_score(Y_test, price_predict), 2))

Mean Squared error:  5.693137123163132
Root Mean Squared error:  2.3860295729858696
Explain variance score for Linear Regression = 0.0
R2 score for Linear Regression = 0.0


In [35]:
test_id = df_te['ID']
test_x= df_te.drop(columns=['ID'])
print(len(test_x.columns))

2


In [36]:
y_prdict = linear_model.predict(test_x)

In [37]:
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['ID', 'item_cnt_month'])
submission.head()

,ID,item_cnt_month
0,0,1.237727
1,1,1.239643
2,2,1.239054
3,3,1.239047
4,4,1.239291


In [38]:
forest_model= RandomForestRegressor(n_estimators = 50, random_state = 42)

# Train the model with training data
forest_model.fit(X_train,Y_train)

#make predictions using random forest model
price_predict =(forest_model.predict(X_test))

forest_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared Error : " , forest_mse)
forest_SqMse = np.sqrt(forest_mse)
print("Root Mean Squared error: ", forest_SqMse)

print("Explain variance score for Random Forest Regression =", round(sm.explained_variance_score(Y_test,price_predict), 2)) 
print("R2 score for Random Forest Regression =", round(sm.r2_score(Y_test, price_predict), 2))

Mean Squared Error :  4.10154338231663
Root Mean Squared error:  2.0252267483708164
Explain variance score for Random Forest Regression = 0.28
R2 score for Random Forest Regression = 0.28


In [40]:
y1_prdict = forest_model.predict(test_x)

In [41]:
submission1 = pd.DataFrame(list(zip(test_id, y1_prdict)),
               columns =['ID', 'item_cnt_month'])
submission1.head()

,ID,item_cnt_month
0,0,1.000000
1,1,1.430435
2,2,1.117140
3,3,1.009859
4,4,1.002222
